# 数组

## 1. 删除排序数组中的重复项 (26)

In [3]:
from typing import List
class Solution:
    def removeDuplicates(self, nums: List[int]) -> int:
        slow = fast = 1
        while fast < len(nums):
            if nums[fast-1] != nums[fast]:
                nums[slow] = nums[fast]
                slow += 1
            fast += 1
        return slow

双指针很容易搞定,试试其他操作

In [6]:
from typing import List
class Solution:
    def removeDuplicates(self, nums: List[int]) -> int:
        s = list(set(nums))
        s.sort()
        ans = len(s)
        for i in range(ans):
            nums[i] = s[i]
        return ans

差不多都是o(n)的时间复杂度，第二种方法的set和list操作不好判断时间代价。空间复杂度双指针是o(1)，第二种最坏应该是nums全不重复，那么就是o(n)的复杂度了。

## 2. 买卖股票的最佳时机 Ⅱ

### 重要的dp

感觉很像回溯之类的算法但是可能比我想的简单，或许是单调栈？

In [43]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        stk = [float("Inf")]
        profit = 0
        for i, num in enumerate(prices):
            if num < stk[-1]:
                if len(stk) > 1: profit += stk[-1] - stk[0]
                stk = [num]
            else: stk.append(num)
        if len(stk) > 1: profit += stk[-1] - stk[0]
        return profit

In [44]:
prices = [7,6,4,3,1]

In [45]:
s = Solution()
s.maxProfit(prices)

0

复杂度： 时间和空间应该都是o(n)

最后题解果然是dp，但是单调栈的思路也和贪心有点像了。

考虑到「不能同时参与多笔交易」，因此每天交易结束后只可能存在手里有一支股票或者没有股票的状态。

定义状态 $dp[i][0]$ 表示第i天交易完后手里没有股票的最大利润， $dp[i][1]$ 表示第i天交易完后手里持有一支股票的最大利润（i从0开始）。

考虑 $dp[i][0]$ 的转移方程，如果这一天交易完成后手里没有股票，那木可能的转移状态为前一天已经没有股票，即 $dp[i-1][0]$ ，或者前一天结束的时候手里持有一支股票，即 $dp[i-1][1]$ ，这时候我们要将其卖出，并获得 $prices[i]$ 的收益。因此为了收益最大化，我们列出如下的转移方程：

$$dp[i][0] = max\{dp[i-1][0], dp[i-1][1] + prices[i]\}$$

再来考虑 $dp[i][1]$ ，按照同样的方式考虑转移状态，那么可能的转移状态为前一天已经持有一支股票，即 $dp[i-1][1]$ ，或者前一天结束时还没有股票，即 $dp[i-1][0]$，这时候我们要将其买入，并减少 $prices[i]$ 的收益，可以列出如下的转移方程：

$$dp[i][1] = max\{ dp[i-1][1], dp[i-1][0] - prices[i] \}$$

对于初始状态，根据状态定义我们可以知道第0天交易结束的时候 $dp[0][0] = 0, dp[0][1] = -prices[0]$ 。

因此，我们只要从前往后依次计算状态即可。由于全部交易结束后，持有股票的收益一定低于不持有股票的收益，因此这时候 $dp[n-1][0]$ 的收益必然是大于 $dp[n-1][1]$ 的，最后的答案即为 $dp[n-1][0]$ 。

In [56]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        dp = [0, -prices[0]]
        print(dp)
        for i in range(1,len(prices)):
            new0 = max(dp[0], dp[1] + prices[i])
            new1 = max(dp[1], dp[0] - prices[i])
            dp = [new0, new1]
            print(dp)
        return dp[0]

In [57]:
prices = [1,2,3,4,5]

In [58]:
s = Solution()
s.maxProfit(prices)

[0, -1]
[1, -1]
[2, -1]
[3, -1]
[4, -1]


4

dp的思路真的强，尤其是简化了dp状态的存储以后。只需要用o(1)的空间。感觉对于dp而言最重要的就是状态转移方程，推导出以后剩下的都很easy

## 3. 旋转数组

### 先整体翻转再部分翻转

In [103]:
class Solution:
    def rotate(self, nums: List[int], k: int) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        def reverse(left, right):
            l = right - left
            for i in range(l//2 + 1):
                nums[left + i], nums[right-i] = nums[right - i], nums[left + i]
            # return nums
        n = len(nums)
        reverse(0, n-1)
        reverse(0, k-1)
        reverse(k, n-1)
        return nums

In [104]:
nums = [1,2,3,4,5,6,7]
k = 3

In [105]:
s = Solution()
s.rotate(nums, k)

[5, 6, 7, 1, 2, 3, 4]

## 4. 存在重复元素

In [ ]:
from collections import defaultdict
class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        cnt = defaultdict(int)
        for n in nums:
            cnt[n] += 1
            if cnt[n] == 2: return True
        return False

o(n), o(n)

In [106]:
class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        return len(set(nums)) != len(nums)

o(n), o(n)

排序后再逐个比较的方法时间复杂度为排序复杂度： o(nlogn)，空间为 o(logn)?

## 5. 只出现一次的数字

### 异或

In [109]:
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        if len(nums) == 1: return nums[0]
        nums.sort()
        for i in range(1, len(nums)-1):
            if nums[i] != nums[i-1] and nums[i] != nums[i+1]: return nums[i]
        if nums[0] != nums[1]: return nums[0]
        if nums[-1] != nums[-2]: return nums[-1]

o(nlogn), o(1)

还是得异或，不过是一块异或而不是两个两个异或

In [121]:
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        ans = 0
        for n in nums: ans ^= n
        return ans

In [122]:
nums = [1,2,2,3,4,3,4]

In [123]:
s = Solution()
s.singleNumber(nums)

1

### reduce()

reduce() 函数会对参数序列中元素进行累积。

reduce(function, iterable[, initializer])

- function -- 函数，有两个参数
- iterable -- 可迭代对象
- initializer -- 可选，初始参数

即对于可迭代对象的第一第二个元素进行func操作，得到的结果再与第三个元素进行func操作。

In [125]:
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        return reduce(lambda x, y: x ^ y, nums)

## 6. 两个数组的交集 Ⅱ

In [ ]:
from collections import Counter
class Solution:
    def intersect(self, nums1: List[int], nums2: List[int]) -> List[int]:
        cnt1 = Counter(nums1)
        cnt2 = Counter(nums2)
        ans = []
        for key in cnt1:
            if key in cnt2:
                l = min(cnt1[key], cnt2[key])
                ans.extend([key] * l)
        return ans

哈希表：
- 时间复杂度：o(m+n)
- 空间复杂度：o(m+n)

排序+双指针：
- 时间复杂度：o(mlogm + nlogn)
- 空间复杂度：o(min(m, n))

## 7. 加一

字符串操作

In [2]:
from typing import List
class Solution:
    def plusOne(self, digits: List[int]) -> List[int]:
        num = ""
        for n in digits:
            num += n
        num = str(int(num) + 1)
        ans = []
        for ch in num:
            ans.append(int(ch))
        return ans

模拟

In [6]:
class Solution:
    def plusOne(self, digits: List[int]) -> List[int]:
        carry = 1
        for i in range(len(digits)-1, -1, -1):
             carry, digits[i] = divmod(digits[i] + carry, 10)
        return digits if not carry else [1] + digits

## 8. 移动零

In [40]:
class Solution:
    def moveZeroes(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        leng = len(nums)
        ptr = 0
        cur = 0
        while ptr < leng:
            while nums[ptr] == 0: ptr += 1
            nums[cur] = nums[ptr]
            cur += 1
            ptr += 1
        for i in range(cur+1, leng): nums[i] = 0

无法处理结尾0的情况

In [54]:
class Solution:
    def moveZeroes(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        leng = len(nums)
        ptr = 0
        for i, n in enumerate(nums):
            if n == 0: continue
            nums[ptr] = n
            ptr += 1
        for i in range(ptr, leng): nums[i] = 0
        return nums

In [59]:
nums = [0]

In [60]:
s = Solution()
s.moveZeroes(nums)

[0]

o(n), o(1)

## 10. 有效的数独

暴力

In [80]:
from collections import defaultdict
class Solution:
    def isValidSudoku(self, board: List[List[str]]) -> bool:
        cnt = defaultdict(list)
        for i in range(9):
            for j in range(9):
                cur = board[i][j]
                x, y = i // 3, j //3
                if cur.isdigit():
                    if cur in cnt[(i, -1)] or cur in cnt[(-1, j)]\
                    or cur in cnt[(x, y)]: return False
                    cnt[(i, -1)].append(cur)
                    cnt[(-1, j)].append(cur)
                    cnt[(x, y)].append(cur)
                # print(cnt)
        return True

In [83]:
board = \
[[".",".",".",".","5",".",".","1","."],
 [".","4",".","3",".",".",".",".","."],
 [".",".",".",".",".","3",".",".","1"],
 ["8",".",".",".",".",".",".","2","."],
 [".",".","2",".","7",".",".",".","."],
 [".","1","5",".",".",".",".",".","."],
 [".",".",".",".",".","2",".",".","."],
 [".","2",".","9",".",".",".",".","."],
 [".",".","4",".",".",".",".",".","."]]

In [82]:
s = Solution()
s.isValidSudoku(board)

False

时间：o(n^2) 空间：o(n^2)

限制在9*9数独的话则都是常数复杂度

## 11. 旋转图像

不会原地

顺时针 90° = 水平翻转 + 转置

In [113]:
class Solution:
    def rotate(self, matrix: List[List[int]]) -> None:
        """
        Do not return anything, modify matrix in-place instead.
        """
        n = len(matrix)
        for i in range(n // 2):
            matrix[i], matrix[n - i - 1] = matrix[n - i - 1], matrix[i]
        for i in range(n):
            for j in range(i):
                print(matrix[i][j], matrix[j][i])
                matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]
        return matrix

In [114]:
matrix = [[5,1,9,11],[2,4,8,10],[13,3,6,7],[15,14,12,16]]

In [115]:
s = Solution()
s.rotate(matrix)

13 14
2 12
4 6
5 16
1 7
9 10


[[15, 13, 2, 5], [14, 3, 4, 1], [12, 6, 8, 9], [16, 7, 10, 11]]

学费了

时间: o(n^2), 空间: o(1)

## 小结：

- 对于数组问题感觉双指针与哈希表比较常用；
- 很多问题在对数组进行排序后都会变得没那么复杂；
- 异或运算对于寻找只出现一次的数字绝杀；
- 要记得一种可以带着数组下标进行排序的思路（870 优势洗牌）；

# 字符串

## 1. 反转字符串

In [117]:
class Solution:
    def reverseString(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        n = len(s)
        for i in range(n // 2):
            s[i], s[n - i - 1] = s[n - i - 1], s[i]
        return s

In [120]:
s = ["H","a","n","n","a","h"]

In [121]:
S = Solution()
S.reverseString(s)

['h', 'a', 'n', 'n', 'a', 'H']

## 2. 整数反转

In [125]:
class Solution:
    def reverse(self, x: int) -> int:
        ans = int(str(abs(x))[::-1])
        if ans > 2**31 - 1: return 0
        return -ans if x < 0 else ans

In [126]:
x = 210

In [127]:
s = Solution()
s.reverse(x)

12

o(n), o(1)

## 3. 字符串中的第一个唯一字符

In [128]:
# 先逃课
from collections import Counter
class Solution:
    def firstUniqChar(self, s: str) -> int:
        cnt = Counter(s)
        for i, ch in enumerate(s):
            if cnt[ch] == 1: return i
        return -1

In [133]:
s = "leetcode"

In [134]:
S = Solution()
S.firstUniqChar(s)

0

o(n), o($\Sigma$) 其中Sigma是字符集

In [142]:
from collections import defaultdict
class Solution:
    def firstUniqChar(self, s: str) -> int:
        cnt = defaultdict(int)
        slow = 0
        for i in range(len(s)):
            cnt[s[i]] += 1
            while cnt[s[slow]] >= 2: 
                slow += 1
                if slow == len(s): return -1
        return slow

In [145]:
s = "aabbccdeeff"

In [146]:
S = Solution()
S.firstUniqChar(s)

6

时间貌似退化为了0(n^2)

### 队列：
- 我们也可以借助队列找到第一个不重复的字符。队列具有「先进先出」的性质，因此很适合用来找出第一个满足某个条件的元素。

In [1]:
from collections import deque
class Solution:
    def firstUniqChar(self, s: str) -> int:
        position = dict()
        q = deque()
        n = len(s)
        for i, ch in enumerate(s):
            # print(position, q)
            if ch not in position:
                position[ch] = i
                q.append((s[i], i))
            else:
                position[ch] = -1
                while q and position[q[0][0]] == -1:
                    q.popleft()
        return -1 if not q else q[0][1]

In [2]:
s = "loveleetcode"

In [3]:
S = Solution()
S.firstUniqChar(s)

2

## 4. 有效的字母异位词

In [4]:
from collections import Counter
class Solution:
    def isAnagram(self, s: str, t: str) -> bool:
        return Counter(s) == Counter(t)

哈希表做法： 时间o(n), 空间o(S)，S为字符集

另可以排序后再判断两字符串是否相等，时间为O(nlogn), 空间为O(logn)

## 5. 验证回文串

In [85]:
class Solution:
    def isPalindrome(self, s: str) -> bool:
        tar = []
        for ch in s:
            if ch.isalnum():
                tar.append(ch.lower())
        n = len(tar)
        for i in range(n//2):
            if tar[i] != tar[n - i - 1]: return False
        return True

In [86]:
s = "0P"

In [87]:
S = Solution()
S.isPalindrome(s)

False

可以原地

In [82]:
class Solution:
    def isPalindrome(self, s: str) -> bool:
        n = len(s)
        if n == 1: return True
        left, right = 0, n - 1
        while left <= right:
            while not s[left].isalnum(): 
                left += 1
                if left == n - 1: return True
            while not s[right].isalnum(): right -= 1
            # print(s[left], s[right])
            if s[left].lower() != s[right].lower(): return False
            left += 1
            right -= 1
        return True

In [83]:
s = " "

In [84]:
S = Solution()
S.isPalindrome(s)

True

**要注意边界以及空字符串的问题。**

这种做法虽然是 O(n)时间 O(1)空间但提交记录还是很慢，难道不是O(n)?

是 O(n)，可能是网络原因。

## 6. 字符串转换整数(atoi)

In [144]:
class Solution:
    def myAtoi(self, s: str) -> int:
        s = s.strip()
        if not s: return 0
        start = 1 if s[0] == "+" or s[0] == "-" else 0
        flag = 0 if s[0] == "-" else 1
        ans = "0"
        for ch in s[start:]:
            if not ch.isdigit(): break
            else: ans += ch
        res = min(int(ans), 2 ** 31 - flag)
        return res if flag else -res

In [145]:
s = "  "

In [146]:
S = Solution()
S.myAtoi(s)

0

时间 O(n), 空间 O(n)

## 8. 外观数列

写一个漂亮的递归

In [155]:
class Solution:
    def countAndSay(self, n: int) -> str:
        def generate(s, n):
            if not n: return s
            leng = len(s)
            s += "0"
            left = 0
            cur = ""
            for i in range(leng):
                if s[i] != s[i+1]:
                    cur += str(i + 1 - left) + s[i]
                    left = i + 1
            return generate(cur, n - 1)
        return generate("1", n-1)

In [160]:
s = Solution()
s.countAndSay(10)

'13211311123113112211'

### 复杂度怎么分析

## 9. 最长公共前缀

In [162]:
from typing import List
class Solution:
    def longestCommonPrefix(self, strs: List[str]) -> str:
        leng = 200
        for s in strs: leng = min(leng, len(s))
        for i in range(leng):
            for s in strs:
                if s[i] != strs[0][i]: return strs[0][:i]
        return strs[0][:leng]

In [180]:
strs = ["flower","fl","flight"]

In [181]:
s = Solution()
s.longestCommonPrefix(strs)

''

- 时间： O(mn) 其中m为strs中最短长度， n为strs长度
- 空间： O(1)

居然写过更简洁的

In [169]:
class Solution:
    def longestCommonPrefix(self, strs: List[str]) -> str:
        res = ""
        for ch in zip(*strs):
            if len(set(ch)) != 1: return res
            res += ch[0]
        return res

以上都是纵向扫描的方式，还有其他方法

横向扫描：

用 $LCP(S_1 ... S_n)$ 表示字符串 $S_1 .. S_n$ 的最长公共前缀。

可以得到以下结论：
$$LCP(S_1 .. S_n) = LCP(LCP(LCP(S_1, S_2), S_3), ... S_n)$$

In [184]:
from functools import reduce
class Solution:
    def longestCommonPrefix(self, strs: List[str]) -> str:
        def LCP(a: str, b: str) -> str:
            if not a: return ""
            n = min(len(a), len(b))
            for i, ch in enumerate(zip(a,b)):
                if ch[0] != ch[1]: return a[:i]
            return a[:n]
        return reduce(LCP, strs)

In [187]:
strs = ["dog","racecar","car"]

In [188]:
s = Solution()
s.longestCommonPrefix(strs)

''

O(mn), O(1)

## 小结：

- 字符串在python下感觉和数组类似，常用的方法如双指针or哈希表在字符串的处理中也经常使用。
- 但字符串与数组最大的不同为：列表在python中为可变对象而   $\quad$     **字符串为不可变对象** $\quad$ ，因此字符串不支持原地对其中的值进行修改，每次改变字符串都要创建一个新的对象，如果考虑内存的话，有的题可能使用列表储存中间变量会好一些。
- 字符串有一个匹配问题的KMP算法，这部分留在LeetCodeStudy中单独学习，这里没有使用KMP解题。

# 链表

## 1. 删除链表中的节点

In [192]:
from typing import Optional
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def deleteNode(self, node: Optional[ListNode]):
        """
        :type node: ListNode
        :rtype: void Do not return anything, modify node in-place instead.
        """
        while node.next.next != None:
            node.val = node.next.val
            node = node.next
        node.val = node.next.val
        node.next = None

试一下递归

In [ ]:
class Solution:
    def deleteNode(self, node: Optional[ListNode]):
        if node.next.next == None:
            node.val = node.next.val
            node.next = None
            return
        node.val = node.next.val
        return self.deleteNode(node.next)

官解真巧妙

In [193]:
class Solution:
    def deleteNode(self, node):
        node.val = node.next.val
        node.next = node.next.next

## 2. 删除链表的倒数第N个节点

In [195]:
class Solution:
    def removeNthFromEnd(self, head: Optional[ListNode], n: int) -> Optional[ListNode]:
        cur = head
        for i in range(n):
            cur = cur.next
        if cur == None: return head.next
        target = head
        while cur.next != None:
            target = target.next
            cur = cur.next
        target.next = target.next.next
        return head

也不是严格意义上的一趟扫描,感觉一趟扫描的做法就只有堆栈，但要使用额外空间。

In [197]:
class Solution:
    def removeNthFromEnd(self, head: ListNode, n: int) -> ListNode:
        dummy = ListNode(0, head)
        stack = list()
        cur = dummy
        while cur:
            stack.append(cur)
            cur = cur.next
        
        for i in range(n):
            stack.pop()

        prev = stack[-1]
        prev.next = prev.next.next
        return dummy.next

### 哑节点的使用：

哑节点能够保证无论如何都不会栈空，同时对付头节点可能被删除的问题。如果被删除的不是头节点，返回哑节点的下一个节点也保证了返回值不受影响。

## 3. 翻转链表

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [5]:
from typing import Optional
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        nodes = list()
        cur = head
        while cur != None:
            nodes.append(cur)
            cur = cur.next
        n = len(nodes)
        for i in range(n//2):
            nodes[i].val, nodes[n - i - 1].val = nodes[n - i - 1].val, nodes[i].val
        return head

可以直接将节点的next指针改为指向前一个节点。

In [7]:
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        if not head: return
        pre, cur = None, head
        while cur.next != None:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        cur.next = pre
        return cur

In [10]:
# 递归
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        if not head: return
        def reverse(pre, cur):
            if not cur.next: 
                cur.next = pre
                return cur
            nxt = cur.next
            cur.next = pre
            return reverse(cur, nxt)
        return reverse(None, head)

迭代与递归时间复杂度都是 O(n)，空间复杂度迭代为 O(1)，递归为 O(n)

## 4. 合并两个有序链表

哨兵节点！

In [14]:
# 迭代
class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        cur = guard = ListNode(0, None)
        while list1 or list2:
            if not list1: 
                cur.next = list2
                break
            if not list2: 
                cur.next = list1
                break
            if list1.val <= list2.val:
                cur.next = list1
                cur = cur.next
                list1 = list1.next
            else:
                cur.next = list2
                cur = cur.next
                list2 = list2.next
        return guard.next

In [17]:
class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        if not list1: return list2
        if not list2: return list1
        if list1.val <= list2.val:
            list1.next = self.mergeTwoLists(list1.next, list2)
            return list1
        else:
            list2.next = self.mergeTwoLists(list1, list2.next)
            return list2

这个递归的重点是参数会随着递归的堆栈而改变，因此最后返回的list1依然是第一层堆栈的list1参数也就是输入的list1链表

## 5. 回文链表

不使用额外空间的关键： 先对后半部分链表进行翻转。

In [20]:
class Solution:
    def isPalindrome(self, head: Optional[ListNode]) -> bool:
        if head.next == None: return True
        guard1 = ListNode(0, None)
        guard1.next = head
        fast = slow = guard1
        while fast != None or fast.next != None:
            slow = slow.next
            fast = fast.next.next
            
        def reverse(pre, cur):
            if not cur.next: 
                cur.next = pre
                return cur
            nxt = cur.next
            cur.next = pre
            return reverse(cur, nxt)
        
        ptr = slow.next = reverse(None, slow.next)
        while ptr != None:
            if head.val != ptr.val: return False
            ptr = ptr.next
            head = head.next
        return True

In [23]:
class Solution:
    def isPalindrome(self, head: Optional[ListNode]) -> bool:
        if head.next == None: return True
        guard1 = ListNode(0, None)
        guard1.next = head
        fast = slow = guard1
        while fast != None and fast.next != None:
            slow = slow.next
            fast = fast.next.next
            
        pre, cur = None, slow.next
        while cur.next != None:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        cur.next = pre
        
        ptr = slow.next = cur
        while ptr != None:
            if head.val != ptr.val: return False
            ptr = ptr.next
            head = head.next
        return True

## 6. 环形链表

In [24]:
class Solution:
    def hasCycle(self, head: Optional[ListNode]) -> bool:
        slow = fast = head
        while fast != None and fast.next != None:
            slow = slow.next
            fast = fast.next.next
            if slow == fast: return True
        return False

### 小结：

- 快慢指针的方法能解决很多链表的问题，包括判断是否有环。
- dummy节点能够很好地处理头节点有可能被改变的问题。
- 回文链表不使用额外空间的方法是先对后半部分反转，类似的思想还出现在旋转数组以及矩阵中。都是先翻转一部分再翻转一部分，结合起来就是最终的变换，可能和矩阵乘法之类的有关，但我证明不了。

# 树

## 1. 二叉树的最大深度

In [25]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [33]:
class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        if not root: return 0
        # if not root.left and not root.right: return 1   # 这句不需要
        left = self.maxDepth(root.left)
        right = self.maxDepth(root.right)
        return max(left, right) + 1

- 时间复杂度： O(n) 
- 空间复杂度： O(H)， 其中H为树的高度，递归需要栈空间。

节点最大深度等于左节点最大深度与右节点最大深度的最大值加一

In [31]:
from collections import deque
class Solution:
    def maxDepth(self, root: Optional[TreeNode]) -> int:
        if not root: return 0
        stk = deque([(1, root)])
        while stk:
            for i in range(len(stk)):
                depth, node = stk.popleft()
                if node.left: stk.append((depth + 1, node.left))
                if node.right: stk.append((depth + 1, node.right))
        return depth

- 时间复杂度： O(n)
- 空间复杂度： O(n)， 需要储存树的节点。

## 2. 验证二叉搜索树

二叉搜索树的中序遍历是一个单调递增序列

In [78]:
from modules import constructTree
a = [5,1,4,None, None, 3,6]
head = constructTree(a)

In [83]:
l = []
def DFS(root):
    if not root: return
    DFS(root.left)
    l.append(root.val)
    return DFS(root.right)

In [84]:
DFS(head)
l

[1, 5, 3, 4, 6]

In [42]:
class Solution:
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        res, stk = [], []
        cur = root
        while stk or cur:
            while cur: 
                stk.append(cur)
                cur = cur.left
            cur = stk.pop()
            if res: 
                if res[-1] >= cur.val: return False
            res.append(cur.val)
            cur = cur.right
        return True

迭代方式：
- 时间复杂度： O(n)
- 空间复杂度： O(n)，需要res和stk储存节点

### 未完成： 中序递归

## 3. 对称二叉树

就是检查两棵二叉树是否相同

In [77]:
class Solution:
    def isSymmetric(self, root: Optional[TreeNode]) -> bool:
        def judge(node1, node2):
            if not node1 and not node2: return True
            if not node1 or not node2: return False
            if node1.val != node2.val: return False
            return judge(node1.left, node2.right) and judge(node1.right, node2.left)
        return judge(root.left, root.right)

递归方式：
- 时间复杂度： O(n)
- 空间复杂度： O(n)

迭代：

首先我们引入一个队列，这是把递归程序改写成迭代程序的常用方法。初始化时我们把根节点入队两次。每次提取两个结点并比较它们的值（队列中每两个连续的结点应该是相等的，而且它们的子树互为镜像），然后将两个结点的左右子结点按相反的顺序插入队列中。当队列为空时，或者我们检测到树不对称（即从队列中取出两个不相等的连续结点）时，该算法结束。

In [87]:
class Solution:
    def isSymmetric(self, root: Optional[TreeNode]) -> bool:
        res = [root, root]
        while res:
            p = res.pop()
            q = res.pop()
            if not p and not q: continue
            if not p or not q: return False
            if p.val != q.val: return False
            res.extend([p.left, q.right, p.right, q.left])
        return True

迭代：
- 时间复杂度： O(n)
- 空间复杂度： O(n)

## 4. 二叉树的层序遍历

In [93]:
from collections import deque
from typing import List
class Solution:
    def levelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
        if not root: return []
        res = []
        stk = deque([root])
        while stk:
            layer = []
            for i in range(len(stk)):
                cur = stk.popleft()
                layer.append(cur.val)
                if cur.left: stk.append(cur.left)
                if cur.right: stk.append(cur.right)
            res.append(layer)
        return res

BFS:
- 时间复杂度： O(n)
- 空间复杂度： O(n)

## 5. 将有序数组转换为二叉搜索树

In [95]:
class Solution:
    def sortedArrayToBST(self, nums: List[int]) -> Optional[TreeNode]:
        def build(left, right):
            if left >= right: return TreeNode(nums[left])
            mid = (left + right) // 2
            root = TreeNode(nums[mid], None, None)
            root.left = build(left, mid - 1)
            root.right = build(mid + 1, right)
            return root
        return build(0, len(nums) - 1)